In [1]:
! pip install transformers 
! pip install datasets 
! pip install huggingface_hub

In [ ]:
# !hugging_face --version

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [28]:
# !apt install git-lfs
# !git config --global user.email "patelkrinal214@gmail.com"
# !git config --global user.name "krinal214"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 75 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 1s (4277 kB/s)  33m

78Selecting previously unselected package git-lfs.
(Reading database ... 103274 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
import transformers
print(transformers.__version__)

4.16.2


In [5]:
model_checkpoint = "bert-base-multilingual-cased"
batch_size = 16

In [6]:
from datasets import load_dataset, load_metric

In [7]:
dataset1=load_dataset('Gautam9595/Squad_Translated')

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/Gautam9595--Squad_Translated-bcd9b1325aec2118/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
datasets=load_dataset("tydiqa","secondary_task")
datasets

Downloading:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/161M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset tydiqa downloaded and prepared to /root/.cache/huggingface/datasets/tydiqa/secondary_task/1.0.0/b8a6c4c0db10bf5703d7b36645e5dbae821b8c0e902dac9daeecd459a8337148. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 49881
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [9]:
import re
def filter_dataset(ds,lang):
    def check(d):
        for i in lang:
            if re.search(i,d):
              return True
        return False
    ds = ds.filter(lambda x: check(x['id']))
    return ds

In [10]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [11]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=8):
    picks = random.sample(range(0, len(dataset)-1), num_examples)
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [12]:
from datasets import concatenate_datasets
final_dataset = concatenate_datasets([dataset1['train'],datasets['train']])
final_dataset= concatenate_datasets([final_dataset,dataset1['validation']])

In [13]:
datasets["train"]=final_dataset
datasets["validation"]=filter_dataset(datasets["validation"],['telugu','bengali'])

  0%|          | 0/6 [00:00<?, ?ba/s]

In [14]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 164436
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 782
    })
})


In [15]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,57293d9e1d046914007791ce,"New_Haven,_Connecticut","New Haven (local /nuː ˈheɪvən/, noo-HAY-vən), in the U.S. state of Connecticut, is the principal municipality in Greater New Haven, which had a total population of 862,477 in 2010. It is located on New Haven Harbor on the northern shore of the Long Island Sound in New Haven County, Connecticut, which in turn comprises the outer limits of the New York metropolitan area. It is the second-largest city in Connecticut (after Bridgeport), with a population of 129,779 people as of the 2010 United States Census. According to a census of 1 July 2012, by the Census Bureau, the city had a population of 130,741.",The population of the Greater New Haven in 2010?,"{'text': ['862,477'], 'answer_start': [164]}"
1,57279ff03acd2414000de830,Jews,"Rates of interreligious marriage vary widely: In the United States, it is just under 50%, in the United Kingdom, around 53%; in France; around 30%, and in Australia and Mexico, as low as 10%. In the United States, only about a third of children from intermarriages affiliate with Jewish religious practice. The result is that most countries in the Diaspora have steady or slightly declining religiously Jewish populations as Jews continue to assimilate into the countries in which they live.[citation needed]",What is the rate of interreligious marriage in Mexico?,"{'text': ['as low as 10%'], 'answer_start': [177]}"
2,5728c5bc4b864d1900164dca,Paris,"Most of the postwar's presidents of the Fifth Republic wanted to leave their own monuments in Paris; President Georges Pompidou started the Centre Georges Pompidou (1977), Valéry Giscard d'Estaing began the Musée d'Orsay (1986); President François Mitterrand, in power for 14 years, built the Opéra Bastille (1985-1989), the Bibliothèque nationale de France (1996), the Arche de la Défense (1985-1989), and the Louvre Pyramid with its underground courtyard (1983-1989); Jacques Chirac (2006), the Musée du quai Branly.",In what year was the Musee de quai Branly built?,"{'text': ['2006'], 'answer_start': [486]}"
3,bengali--4869052194737042764-8,অ্যানড্রয়েড (অপারেটিং সিস্টেম),"৫ই নভেম্বর, ২০০৭ সালে ওপেন হ্যান্ডসেট এল্যায়েন্স সূচনা করে যাতে ছিল ব্রডকম কর্পোরেশন, গুগল, এইচটিসি, ইন্টেল, এলজি, মার্ভেল টেকনোলজি গ্রুপ, মটোরোলা, এনভিডিয়া, কোয়ালকম, স্যামস্যাং ইলেক্ট্রনিকস, স্প্রিন্ট নেক্সটেল, টি-মোবাইল এবং টেক্সাস ইনস্ট্রুমেন্ট। ওপেন হ্যান্ডসেট এল্যায়েন্সের উদ্দেশ্য হল মুক্ত ধরনের মোবাইল হ্যান্ডসেট প্লাটফর্ম তৈরী করা[9]। একই দিনে, ওপেন হ্যান্ডসেট এল্যায়েন্স তাদের প্রথম পণ্য অ্যানড্রয়েড ছাড়ে যা লিনাক্স কারনেল ২.৬.২৫ এর উপর ভিত্তি করে তৈরী[9][36]। প্রথম বাণিজ্যিকভাবে ছাড়া স্মার্টফোনটি ছিল এইচটিসি ড্রিম যা ২০০৮ সালের ২২ অক্টোবর ছাড়া হয়।[37]",কোন কোম্পানির মোবাইলে সর্বপ্রথম অ্যানড্রয়েড বা এন্ড্রয়েড ব্যবহার করা হয় ?,"{'text': ['এইচটিসি'], 'answer_start': [521]}"
4,5706076875f01819005e78b0,The_Times,"The Saturday edition of The Times contains a variety of supplements. These supplements were relaunched in January 2009 as: Sport, Weekend (including travel and lifestyle features), Saturday Review (arts, books, and ideas), The Times Magazine (columns on various topics), and Playlist (an entertainment listings guide).",An entertainment listings guide in the Saturday edition of The Times is called what?,"{'text': ['Playlist'], 'answer_start': [275]}"
5,572846ff2ca10214002da22e,Labour_Party_(UK),"After a debate, the 129 delegates passed Hardie's motion to establish ""a distinct Labour group in Parliament, who shall have their own whips, and agree upon their policy, which must embrace a readiness to cooperate with any party which for the time being may be engaged in promoting legislation in the direct interests of labour."" This created an association called the Labour Representation Committee (LRC), meant to coordinate attempts to support MPs sponsored by trade unions and represent the working-class population. It had no single leader, and in the absence of one, the Independent Labour Party no

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [17]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 160  # The allowed overlap between two part of the context when splitting is performed.

In [18]:
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1 :
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
    
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [19]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

  0%|          | 0/165 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [32]:
from transformers import TrainingArguments, Trainer
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-model",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    #push_to_hub = True,
    #push_to_hub_model_id = "augmented_Squad_Translated",
    save_steps=10834,

)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.7/site-packages/transformers/training_args.py:951: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case krinal214/augmented_Squad_Translated).
  FutureWarning,


In [33]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [34]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/kaggle/working/bert-base-multilingual-cased-finetuned-model is already a clone of https://huggingface.co/krinal214/augmented_Squad_Translated. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [35]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 173350
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 930
    })
})


In [36]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 173350
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10835


Epoch,Training Loss,Validation Loss
1,1.115400,0.525094


Saving model checkpoint to bert-base-multilingual-cased-finetuned-model/checkpoint-10834
Configuration saved in bert-base-multilingual-cased-finetuned-model/checkpoint-10834/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-model/checkpoint-10834/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/checkpoint-10834/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/checkpoint-10834/special_tokens_map.json
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

***** Running Evaluation *****
  Num examples = 930
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=10835, training_loss=1.2931404511610594, metrics={'train_runtime': 7151.7967, 'train_samples_per_second': 24.239, 'train_steps_per_second': 1.515, 'total_flos': 3.39718445851392e+16, 'train_loss': 1.2931404511610594, 'epoch': 1.0})

In [37]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [38]:
max_answer_length = 50
n_best_answers=20

In [39]:
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 50):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[start_index])==0
                        or len(offset_mapping[end_index])==0
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    try:
                        start_char = offset_mapping[start_index][0]
                        end_char = offset_mapping[end_index][1]
                        valid_answers.append(
                            {
                                "score": start_logits[start_index] + end_logits[end_index],
                                "text": context[start_char: end_char]
                            }
                        )
                    except IndexError:
                        continue

        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]
        
    return predictions


In [40]:
import collections
def evaluate(ds,wrong_pred,language):
    val_dataset=filter_dataset(ds,language)
    validation_features = val_dataset.map(
        prepare_validation_features,
        batched=True,
        remove_columns=val_dataset.column_names
    )
    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
    
    examples = val_dataset
    features = validation_features

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    final_predictions = postprocess_qa_predictions(val_dataset, validation_features, raw_predictions.predictions,n_best_answers,max_answer_length)
    metric = load_metric("squad")
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in val_dataset]
    for j in range(len(references)):
        wrong=True
        for t in references[j]["answers"]["text"]:
            if normalize_answer(formatted_predictions[j]['prediction_text'])==normalize_answer(t):
                wrong=False
                break
        if wrong:
            wrong_pred.append({"id":references[j]["id"],"prediction":formatted_predictions[j]["prediction_text"],"original:":t})
    return metric.compute(predictions=formatted_predictions, references=references)

In [41]:
import string
eval_languages = [['bengali'], ['telugu']]
wrong_prediction={}
wrong_prediction['bengali']=[]
wrong_prediction['telugu']=[]
wrong_prediction['all']=[]
for lang in eval_languages:
    output = evaluate(datasets["validation"],wrong_prediction[lang[0]],lang)
    print(lang[0],':',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 144
  Batch size = 16


Post-processing 113 example predictions split into 144 features.


  0%|          | 0/113 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 786
  Batch size = 16


Post-processing 669 example predictions split into 786 features.


  0%|          | 0/669 [00:00<?, ?it/s]

telugu : {'exact_match': 71.30044843049328, 'f1': 84.07107605604045}


In [42]:
output = evaluate(datasets["validation"],wrong_prediction['all'],['telugu','bengali'])
print('overall',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 930
  Batch size = 16


Post-processing 782 example predictions split into 930 features.


  0%|          | 0/782 [00:00<?, ?it/s]

overall {'exact_match': 71.35549872122762, 'f1': 83.4477309962491}


In [43]:
print('wrong predictions(bengali)',len(wrong_prediction['bengali']))
print('wrong predictions(telugu)',len(wrong_prediction['telugu']))


wrong predictions(bengali) 32
wrong predictions(telugu) 192


In [45]:
picks = random.sample(range(0, len(wrong_prediction['bengali'])-1), 30)
for i in picks:
    print(wrong_prediction['bengali'][i])

{'id': 'bengali-4841554072514664155-0', 'prediction': 'ইন্দিরা গান্ধী', 'original:': 'শেখ মুজিবুর রহমান'}
{'id': 'bengali-4980664758092265474-1', 'prediction': 'ব্রহ্মা', 'original:': 'সাং চিয়েন'}
{'id': 'bengali--6695334746728483406-2', 'prediction': 'ঝাঁসির রানি ঝাঁসির রানির', 'original:': 'ঝাঁসির রানি'}
{'id': 'bengali-4713184652126684859-6', 'prediction': 'মিত্র কৃষ্ণ', 'original:': 'কৃষ্ণ'}
{'id': 'bengali-6608660370621955794-0', 'prediction': 'ইন্দো-আর্য', 'original:': 'খাড়ি বোলি উপভাষা'}
{'id': 'bengali--8422112448971508966-2', 'prediction': '১০৫টি', 'original:': '১০৫'}
{'id': 'bengali--8404986622637348674-3', 'prediction': 'পাকিস্তান বনাম আইসিসির নতুন সদস্য শ্রীলঙ্কার', 'original:': 'ভারত'}
{'id': 'bengali--9113522782624640859-1', 'prediction': 'চট্টগ্রাম বিশ্ববিদ্যালয়', 'original:': 'চট্টগ্রাম'}
{'id': 'bengali--7381837526075378596-1', 'prediction': 'চতুর্থ থেকে দ্বাদশ শতাব্দীর মধ্যে', 'original:': 'চতুর্থ থেকে দ্বাদশ শতাব্দী'}
{'id': 'bengali-4667499823130888007-3', 'predi

In [46]:
picks = random.sample(range(0, len(wrong_prediction['telugu'])-1), 40)
for i in picks:
    print(wrong_prediction['telugu'][i])

{'id': 'telugu-918133519643966911-0', 'prediction': 'పూణే', 'original:': 'పూణే, మహారాష్ట్ర'}
{'id': 'telugu-2355058992665135572-0', 'prediction': '198 హెక్టార్లలో', 'original:': '198 హెక్టార్ల'}
{'id': 'telugu--5347969429403013814-1', 'prediction': '160 చదరపు కిలోమీటర్ల', 'original:': '2 చదరపు మైళ్ళు)'}
{'id': 'telugu--6194068905241325426-0', 'prediction': '361 హెక్టార్లలో', 'original:': '361 హెక్టార్ల'}
{'id': 'telugu-997773189427554722-0', 'prediction': '282 హెక్టార్లలో', 'original:': '282 హెక్టార్ల'}
{'id': 'telugu-4831434226867109360-7', 'prediction': '53 సంవత్సరాల వయసులో 1915 నవంబర్ 30', 'original:': '53'}
{'id': 'telugu-7280221406834583430-1', 'prediction': 'అలి గడ్', 'original:': 'ఢిల్లి'}
{'id': 'telugu--3406235526541671609-0', 'prediction': '671 హెక్టార్లలో', 'original:': '671 హెక్టార్ల'}
{'id': 'telugu-7783818474710184169-13', 'prediction': 'అనంత్, ఆకాష్,\xa0ఒక\xa0కూతురు\xa0ఇషా', 'original:': 'ఇద్దరు\xa0కొడుకులు\xa0అనంత్, ఆకాష్,\xa0ఒక\xa0కూతురు\xa0ఇషా'}
{'id': 'telugu--331693

In [47]:
#trainer.push_to_hub()

Saving model checkpoint to bert-base-multilingual-cased-finetuned-model
Configuration saved in bert-base-multilingual-cased-finetuned-model/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-model/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/676M [00:00<?, ?B/s]

Upload file runs/Apr08_15-59-14_afed8660f74f/events.out.tfevents.1649433564.afed8660f74f.34.2: 100%|##########…

To https://huggingface.co/krinal214/augmented_Squad_Translated
   d070705..d24e18e  main -> main



...

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/krinal214/augmented_Squad_Translated
   d24e18e..af6b682  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/krinal214/augmented_Squad_Translated/commit/d24e18e06aa0858e37d8cdf361f3143a07fd382d'